# <font color='blue'>Data Pipeline using PySpark
---

**Outline**

1. Review:
    - Introduction to Apache Spark & PySpark I
    - Introduction to Apache Spark & PySpark II
2. Case Study: Data Pipeline Movie Data

### **Create Spark Session**
---

We're going to name the session as `"Data Pipeline using PySpark"`

In [1]:
# import library
import pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .appName("Data Pipeline using PySpark") \
    .getOrCreate()

In [4]:
spark

### **1. Extract Process**
---

#### **1a**
---

- Read the csv data first in `data/ratings.csv` using PySpark
- Save it to `df_ratings` variable

In [5]:
def read_data_csv(PATH:str,filename:str):
    try:
        """
            Function to read data from csv file 
        """
        data = spark.read.option("header", "true").csv(DATA_PATH + filename) 
        return data
    except Exception as e:
        print(f"ERROR : {e}")    

In [6]:
DATA_PATH = '../data/'
filename = 'ratings.csv'
df_ratings = read_data_csv(DATA_PATH, filename)
df_ratings.show(3)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    110|   1.0|1425941529|
|     1|    147|   4.5|1425942435|
|     1|    858|   5.0|1425941523|
+------+-------+------+----------+
only showing top 3 rows



In [14]:
df_ratings.count()

26024289

#### **1b**
---

In [7]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [8]:
from dotenv import load_dotenv
import os

In [9]:
# Load .env file
load_dotenv()

# Access variables
db_host = os.getenv("POSTGRES_URL_MOVIE")
db_name = os.getenv("POSTGRES_DB_MOVIE")
db_user = os.getenv("POSTGRES_USER_MOVIE")
db_pass = os.getenv("POSTGRES_PASSWORD_MOVIE")
conn_properties = {
    "user": db_user,
    "password": db_pass,
    "driver": "org.postgresql.Driver" # set driver postgres
}

In [10]:
TABLE_NAME = 'movies_metadata'
df_metadata = spark.read.jdbc(url=db_host,table = TABLE_NAME, properties=conn_properties)

In [15]:
df_metadata.show(3, truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 adult                 | False                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [13]:
df_metadata.count()

45466

### **2. Source to Target Mapping**
---

- After we extracted all the data, we got information that ratings and movie metadata can be joined by using the movie id
- Before that, we must profile our table that can help us for the next step
- You can create your Source to Target Mapping or you can see it on this [Spreadsheet](https://docs.google.com/spreadsheets/d/1spFcpnUdoiKW2dApInxDNkadywiBs2mdCt8CmwHZ1ME/edit?usp=sharing)
- The goals of Source to Target Mapping is to identify which columns can be used for the next process like what transformation rules that can be used

### **3. Transform Data**
---

- After we mapping all the columns from Source to Target Mapping, now we can set the transformation rules
- These are the transformation processes that we will do in this case:
    - Join Data
    - Renaming Columns
    - Select Data based on Columns
    - Casting Data Type
    - Filter Data
    - Create New Columns using Existing Columns

#### **3a. Join Data**
---

- In this process, we want to join the data from movies rating and movies metadata
- After we explore the data from the previous process turns out there's one column that we can join!
    - `df_ratings`: `movieId`
    - `df_metadata`: `id`

    <br>
    <center>
        <img src="https://sekolahdata-assets.s3.ap-southeast-1.amazonaws.com/notebook-images/mde-data-ingestion-spark/join_data_case_week_5.png" width=50%>
    </center>
    <br>

- To join the data using PySpark, we can use this code

```python
df1.join(df2, df1.col == df2.col, JOIN_METHOD) # we can use inner, left, right, etc
```

- In this case, we will use `inner` join then save the joined data in `df_joined` variable